In [ ]:
(* #use "topfind";; *)
(* #require "ppx_deriving";; *)
(* #require "ppx_jane";; *)
#require "graphics";;
#require "yojson";;
#require "ppx_yojson_conv";;
(* #require "core";; *)
#require "async";;
#require "cohttp-async";;
#require "async_ssl";;
#require "ppx_let" ;;

#require "owl-top, jupyter.notebook";;
#require "jupyter-archimedes";;
#require "owl-jupyter";;
#require "owl-plplot";;

(* open Owl_jupyter;; *)
(* open Owl;; *)

(* #thread ;;
let () = Thread_safe.block_on_async_exn (fun () -> get_token "15011272962" "u123456A") *)

In [2]:
#require "k98";;

/Users/faotbgs/.opam/4.12.0/lib/k98: added to search path
/Users/faotbgs/.opam/4.12.0/lib/k98/k98.cma: loaded


In [3]:
let token = K98.JQ.get_token "15011272962" "u123456A"

val token : string = "5b6a9ba2b7f07fb129667f2f04ca0fb9cd71f765"


In [7]:
let code = "002407.XSHE"
let ticks = K98.JQ.get_ticks token code 100 "2021/11/23 00:00:00"

val code : string = "002407.XSHE"


error: runtime_error